In [1]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from crud import AnimalShelter

###########################
# Data Manipulation / Model
###########################
# FIX ME update with your username and password and CRUD Python module name

username = "aacuser"
password = "Mak3m3R1ch"
host = "nv-desktop-services.apporto.com"
port = 32774
database_name = "AAC"

# Connect to database via CRUD Module
db = AnimalShelter(username, password, host, port, database_name)

df = pd.DataFrame.from_records(db.read({}))
if '_id' in df.columns:
    df.drop(columns=['_id'], inplace=True)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
#df = pd.DataFrame.from_records(db.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
#df.drop(columns=['_id'],inplace=True)

## Debug
#print(len(df.to_dict(orient='records')))
#print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'Grazioso Salvare Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#Dashboard Layout
app.layout = html.Div([
    #html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('Erin Zamora - SNHU - CS-340 Dashboard'))),
    html.Center(html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()), style={'width': '500px'})),
    html.Label('Filter by Rescue Type:'),
    dcc.Dropdown(
        id='filter-type',
        options=[
            {'label': 'All', 'value': 'All'},
            {'label': 'Water Rescue', 'value': 'Water Rescue'},
            {'label': 'Mountain or Wilderness Rescue', 'value': 'Mountain or Wilderness Rescue'},
            {'label': 'Disaster or Individual Tracking', 'value': 'Disaster or Individual Tracking'}
        ],
        value='All',
        style={'width': '60%'}
    ),
    html.Hr(),
    html.Div(
        
#Interactive drop down Filter

    ),
    html.Hr(),
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),
        editable=True,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable="single",
        row_selectable="multi",  # Keep this for multi-row selection
        selected_columns=[],
        selected_rows=[],
        page_action="native",
        page_current=0,
        page_size=10
),

                         
#Interactive DataTable

    html.Br(),
    html.Hr(),
    
    html.Div(
        children=[
            html.Div(id='graph-id', style={'width': '50%', 'display': 'inline-block', 'vertical-align': 'top'}),
            html.Div(id='map-id', style={'width': '50%', 'display': 'inline-block', 'vertical-align': 'top'})
        ],
        style={'display': 'flex'}
    )
])

#############################################
# Interaction Between Components / Controller
#############################################

    
@app.callback(Output('datatable-id','data'),
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
    query = {}
    if filter_type == 'Water Rescue':
        query = {"breed": {"$in": ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]}}
    elif filter_type == 'Mountain or Wilderness Rescue':
        query = {"breed": {"$in": ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"]}}
    elif filter_type == 'Disaster or Individual Tracking':
        query = {"breed": {"$in": ["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiler"]}}
    
    filtered_df = pd.DataFrame.from_records(db.read(query))
    if '_id' in filtered_df.columns:
        filtered_df.drop(columns=['_id'], inplace=True)
    return filtered_df.to_dict('records')

# Display the breeds of animal based on quantity represented in
# the data table
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])
def update_graphs(viewData):
    df = pd.DataFrame(viewData)
    if df.empty:
        return "No data to display"
    fig = px.pie(df, names='breed', title='Breed Distribution')
    return dcc.Graph(figure=fig)
    
#This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):  
    # Convert the input data to a DataFrame
    dff = pd.DataFrame.from_dict(viewData)

    # Handle missing or empty data
    if dff.empty:
        return html.Div("No data available to display on the map.", style={'color': 'red', 'font-weight': 'bold'})

    # Safely handle missing or empty selected rows
    if not index or index[0] is None:
        row = 0  # Default to the first row if no selection
    else:
        row = index[0]

    # Handle missing or null values gracefully
    try:
        lat = dff.iloc[row].get("location_lat", 30.75)  # Default latitude for Austin, TX
        lon = dff.iloc[row].get("location_long", -97.48)  # Default longitude for Austin, TX
        breed = dff.iloc[row].get("breed", "Unknown Breed")
        name = dff.iloc[row].get("name", "No Name Available")

        # Fallback for missing or NaN coordinates
        lat = lat if pd.notna(lat) else 30.75
        lon = lon if pd.notna(lon) else -97.48

    except (IndexError, KeyError):
        # Fallback values in case of errors
        lat, lon = 30.75, -97.48
        breed, name = "Unknown Breed", "No Name Available"

    # Return map with the marker
    return [
        dl.Map(
            style={'width': '100%', 'height': '500px'},
            center=[lat, lon],
            zoom=10,
            children=[
                dl.TileLayer(id="base-layer-id"),
                dl.Marker(
                    position=[lat, lon],
                    children=[
                        dl.Tooltip(breed),
                        dl.Popup([
                            html.H4("Animal Name"),
                            html.P(name)
                        ])
                    ]
                )
            ]
        )
    ]

        
    

app.run_server(debug=True)


Connected to database: AAC, collection: animals
Dash app running on http://127.0.0.1:27306/
